In [1]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import time 
from sklearn import metrics

In [3]:
# Parameters
learning_rate = 0.01
epochs = 10
batch_size = 32


# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [4]:
# Create model
def multilayer_perceptron(x, n_input, n_hidden_1, n_classes):
    # Hidden layer with RELU activation
    with tf.name_scope("hidden_layer"):
        w1= tf.Variable(tf.random_normal([n_input, n_hidden_1]), name = "weights")
        layer_1 = tf.nn.sigmoid(tf.matmul(x,w1))
    # Output layer with linear activation
    with tf.name_scope("output_layer"):
        w_o = tf.Variable(tf.random_normal([n_hidden_1, n_classes]), name = "weights")
        out_layer = tf.matmul(layer_1, w_o)
    return out_layer

In [5]:
# Construct model
pred = multilayer_perceptron(x, n_input, n_hidden_1, n_classes)

In [6]:
# Define loss and optimizer
with tf.name_scope("cross_entropy"):
    out = tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y)
    with tf.name_scope("total_loss"):
        cross_entropy = tf.reduce_mean(out)
tf.summary.scalar("cross_entropy", cross_entropy)

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
    
with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [7]:
# We can see all the operations embedded to the graph. You can see how scope helps in naming an operation at the end.
sess = tf.Session()
op = sess.graph.get_operations()
[m.values() for m in op]

[(<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>,),
 (<tf.Tensor 'Placeholder_1:0' shape=(?, 10) dtype=float32>,),
 (<tf.Tensor 'hidden_layer/random_normal/shape:0' shape=(2,) dtype=int32>,),
 (<tf.Tensor 'hidden_layer/random_normal/mean:0' shape=() dtype=float32>,),
 (<tf.Tensor 'hidden_layer/random_normal/stddev:0' shape=() dtype=float32>,),
 (<tf.Tensor 'hidden_layer/random_normal/RandomStandardNormal:0' shape=(784, 256) dtype=float32>,),
 (<tf.Tensor 'hidden_layer/random_normal/mul:0' shape=(784, 256) dtype=float32>,),
 (<tf.Tensor 'hidden_layer/random_normal:0' shape=(784, 256) dtype=float32>,),
 (<tf.Tensor 'hidden_layer/weights:0' shape=(784, 256) dtype=float32_ref>,),
 (<tf.Tensor 'hidden_layer/weights/Assign:0' shape=(784, 256) dtype=float32_ref>,),
 (<tf.Tensor 'hidden_layer/weights/read:0' shape=(784, 256) dtype=float32>,),
 (<tf.Tensor 'hidden_layer/MatMul:0' shape=(?, 256) dtype=float32>,),
 (<tf.Tensor 'hidden_layer/Sigmoid:0' shape=(?, 256) dtype=float32>,),
 (

In [8]:
sess.close()

In [9]:
logs_path = '/tmp/tensorflowlogs/datagiri_nn_tf_1'

In [10]:
# Initializing the variables
init = tf.global_variables_initializer()

merged_summary_op = tf.summary.merge_all()

In [11]:
start = time.time()
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph = sess.graph)
    for i in range(epochs):
        batches = int(len(mnist.train.images)/batch_size)
        for j in range(batches):
            _, summary = sess.run([optimizer, merged_summary_op], feed_dict={x: mnist.train.images[j*batch_size:(j+1)*batch_size,:],
                                                          y: mnist.train.labels[j*batch_size:(j+1)*batch_size,:]})
            summary_writer.add_summary(summary, epochs*batches+i)
            if j % 500 == 0:
                train_acc = sess.run([accuracy], feed_dict={x: mnist.train.images, y: mnist.train.labels})
                valid_acc = sess.run([accuracy], feed_dict={x: mnist.validation.images, y: mnist.validation.labels})
                print ("epoch: ", i , "Train_Accuracy: ", train_acc, "Validation_Accuracy: ", valid_acc, "time_taken:", (time.time() - start))
    print ("total time taken: ", time.time()-start)
    test_accuracy = sess.run([accuracy],feed_dict={x: mnist.test.images, y: mnist.test.labels} )
    print("Test Accuracy:", test_accuracy)

epoch:  0 Train_Accuracy:  [0.19354546] Validation_Accuracy:  [0.18260001] time_taken: 1.088667869567871
epoch:  0 Train_Accuracy:  [0.90918183] Validation_Accuracy:  [0.90899998] time_taken: 3.239824056625366
epoch:  0 Train_Accuracy:  [0.93229091] Validation_Accuracy:  [0.9278] time_taken: 5.385020971298218
epoch:  0 Train_Accuracy:  [0.94660002] Validation_Accuracy:  [0.94279999] time_taken: 7.519505977630615
epoch:  1 Train_Accuracy:  [0.94390911] Validation_Accuracy:  [0.94340003] time_taken: 8.866734027862549
epoch:  1 Train_Accuracy:  [0.95645458] Validation_Accuracy:  [0.94859999] time_taken: 10.993315935134888
epoch:  1 Train_Accuracy:  [0.95538181] Validation_Accuracy:  [0.94760001] time_taken: 13.170160055160522
epoch:  1 Train_Accuracy:  [0.96267271] Validation_Accuracy:  [0.95859998] time_taken: 15.31642198562622
epoch:  2 Train_Accuracy:  [0.96294546] Validation_Accuracy:  [0.9562] time_taken: 16.691431999206543
epoch:  2 Train_Accuracy:  [0.9667818] Validation_Accuracy: 

In [12]:
! tensorboard --logdir=/tmp/tensorflowlogs/datagiri_nn_tf_1

ERROR:tensorflow:Tried to connect to port 6006, but address is in use.
Tried to connect to port 6006, but address is in use.
